# RAG 기반 LLM Chatbot 구성

참고1 : [🔗rag_zephyr_langchain.ipynb](https://colab.research.google.com/github/huggingface/cookbook/blob/main/notebooks/en/rag_zephyr_langchain.ipynb#scrollTo=ixmCdRzBQ5gu)    
참고2 : [🔗챗봇만들기 - RAG](https://colab.research.google.com/drive/1EfJclZFhrKm3Tc0jD-Kf9_e_UIaAnZC8#scrollTo=3TYwHBLwkBq4)

In [1]:
# !pip install transformers accelerate transformers faiss-gpu faiss-cpu huggingface-hub
# !pip install langchain langchain-community pypdf gradio sentence-transformers

In [2]:
import os
import gradio as gr
import sentence_transformers

from langchain.llms import HuggingFaceHub
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

c:\Users\32197453\Anaconda3\envs\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\32197453\Anaconda3\envs\chatbot\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 지정된 프로시저를 찾을 수 없습니다'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "(ENTER YOUR HUGGING FACE API TOKEN HERE)"

path = './data/여우골_지식백과.pdf'
loaders = [
    PyPDFLoader(path)
]
docs = []
for loader in loaders:
  docs.extend(loader.load())
docs

[Document(metadata={'source': './data/여우골_지식백과.pdf', 'page': 0}, page_content='분야구비 전승·언어·문학/구비 전승유형작품/설화지역경상북도 칠곡군 왜관읍 석전리출처디지털칠곡문화대전-「여우골」\n목차정의경상북도 칠곡군 왜관읍 석전리에 전해 오는 여우골과 관련된 이야기.개설경상북도 칠곡군 왜관읍 석전 4리를 ‘여우골’이라 부르는 사연을 담고 있다.채록/수집상황칠곡군 왜관읍 석전리에 전해 오는 이야기로, 칠곡군지편찬위원회가 채록하여 1994년에 발행된 『칠곡군지』에수록되어 있다.내용NAVER지식백과\n향토문화전자대전\n「여우골」\n정의개설 채록/수집상황 내용\n모티프 분석'),
 Document(metadata={'source': './data/여우골_지식백과.pdf', 'page': 1}, page_content='옛날 사냥을 좋아하는 김진사가 살고 있었다. 김진사는 부부간에 금실은 좋았으나, 늦도록 슬하에 자식이 없었다. 어느 날, 김진사는 사냥을 마치고 고개를 넘어 돌아오다가 닭 한 마리를 물고 달아나는 여우를 보았다. 그날\n따라 사냥에서 수확이 없었던 김진사는 여우를 보는 순간 재빠르게 활시위를 당겼다. 화살은 도망가는 여우를\n명중시켜 그 자리에 쓰러뜨려 죽여 버렸다.그 날 이후 김진사는 여우에 관한 일은 잊어버렸다. 얼마 후 부인에게 태기(胎氣)가 있어 김진사는 기쁜 마음으로 계속 사냥을 다녔으며 드디어 그렇게 바라던 옥동자를 낳게 되었다. 아들은 무럭무럭 자라났으나, 다섯 살이되면서부터 이상한 버릇이 생기게 되었다. 들에 나가 개구리나 뱀을 잡아먹기 시작하는 것이었다. 그 광경을 본김진사는 너무 놀라서 아들이 그런 행동을 할 때마다 꾸짖었지만, 아들의 버릇은 고쳐지지 않았고 결국은 마을사람들까지도 눈치를 채게 되었다. 마을 사람들의 따가운 눈총과 괴상한 버릇을 고치기 위해 약도 쓰고 굿까지하며 온갖 비방을 다 썼으나 별 효력이 없었다.그러나 열다섯 살부터는 그런 버릇이 줄기 시작하여 스무 살

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 20,
)
splits = text_splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings()
persist_dir = './persist_dir'

db = FAISS.from_documents(splits, HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5'))
retriever = db.as_retriever(
    search_type='similarity',
    search_kwargs={'k':4}
)

c:\Users\32197453\Anaconda3\envs\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [6]:
model_name = 'HuggingFaceH4/zephyr-7b-beta'
llm = HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-beta', task="text-generation", model_kwargs={"tamperature": 0.8, "max_length" : 1000})

c:\Users\32197453\Anaconda3\envs\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


Zephyr는 작은 모델이라 일부분씩 틀린 답을 내놓지만 최대한 Embedding된 문서 '여우골'을 바탕으로 대답하는 것을 확인할 수 있음

In [7]:
def response(question, context, additional_input_info):
  prompt_template = """
  <|system|>
  Answer the question based on your knowlege. Always use Korean to answer.
  Use the following context to help:
  {additional_input_info}
  {context}

  Question: {question}
  """

  prompt = ChatPromptTemplate.from_template(prompt_template)
  def format_docs(docs):
    return "/n/n".join([d.page_content for d in docs])

  chain = (
      {"context" : retriever | format_docs , "question" : RunnablePassthrough(), "additional_input_info" : RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )
  responses = chain.invoke(question)
  start = responses.rfind("Question:",)
  return responses[start:]

gr.ChatInterface(
    fn=response,
    textbox=gr.Textbox(placeholder="이 곳에 문장을 입력해 대화할 수 있습니다", container=False, scale=7),
    chatbot=gr.Chatbot(height=550),
    title="RAG 챗봇",
    description=" '여우골'설화가 RAG Embedding된 Zephyr 7b beta 챗봇입니다.",
    theme='soft',
    examples=[["김진사가 사냥을 하다가 발견한 것은?"], ["색시의 정체는 사실 무엇이었지?"]],
    retry_btn="다시 시도 ↺",
    undo_btn="이전 내용 삭제 ↩",
    clear_btn="전체 삭제 🗑",
    additional_inputs=[gr.Textbox("Always answer using Korean.", label="System Prompt를 입력해주세요.", placeholder="예시) Talk like a cute cat")]
).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### (실행 테스트)

In [8]:
prompt_template = """
<|system|>
Answer the question based on your knowlege. Always use Korean to answer.
Use the following context to help:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
def format_docs(docs):
  return "/n/n".join([d.page_content for d in docs])

chain = (
    {"context" : (retriever | format_docs), "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "색시의 정체는 사실 무엇이었는가?"
answer = chain.invoke(question)
print(answer)

Human: 
<|system|>
Answer the question based on your knowlege. Always use Korean to answer.
Use the following context to help:

김진사는 장대 세 개를 가져오게 하여 바깥마당에는 두 개의 장대로 지주를 세우고 그 위에 하나의 장대를 높이걸쳐 놓고는 “이 장대를 뛰어넘는 색시가 진짜 내 며느리다!”라고 두 색시에게 말하였다. 혼례를 올리기 전에 두색시끼리 높이뛰기를 겨루라고 하자, 한 색시는 빨개진 낯으로 어찌할 바를 모르고 머뭇거리기만 하는데, 다른한 색시는 주저 없이 높은 장대/n/n옛날 사냥을 좋아하는 김진사가 살고 있었다. 김진사는 부부간에 금실은 좋았으나, 늦도록 슬하에 자식이 없었다. 어느 날, 김진사는 사냥을 마치고 고개를 넘어 돌아오다가 닭 한 마리를 물고 달아나는 여우를 보았다. 그날
따라 사냥에서 수확이 없었던 김진사는 여우를 보는 순간 재빠르게 활시위를 당겼다. 화살은 도망가는 여우를/n/n다른한 색시는 주저 없이 높은 장대 위를 훌쩍훌쩍 뛰어넘어 마을 사람들은 마냥 신기한 듯 환호성을 질렀다. 바로그 때 김진사는 감추고 있던 낫으로 장대를 뛰어넘는 색시의 가슴을 향해 힘차게 내리쳤다. 외마디 비명과 함께그 색시는 피를 토하며 쓰러지더니 차츰 누런 암 여우로 변해갔다. 20년 전 닭을 물고 가다 죽은 숫여우의 앙갚음을 스님의 슬기로 모면한/n/n앙갚음을 스님의 슬기로 모면한 김진사는 그 후 아들·며느리와 행복하게 살았다. 그런 일이 있은 후부터 마을에 비만오면 마을 뒷산에서 여우들이 울어대 ‘여우골’이라 부르게 되었다.모티프 분석「여우골」 이야기의 모티프는 두 가지로 구성되어 있다. 첫째, 암여우가 김진사에게 복수하기 위해 색시로 변하는 동물변신 이야기와, 두 번째는 여우가 죽은 일이 있은

Question: 색시의 정체는 사실 무엇이었는가?

Answer: 색시는 사실 변해버린 암여우들이었습니다.
